In [1]:
import pandas as pd
import s3fs

In [8]:
# fichiers représentants
def import_inpi():
    """_summary_
    Function that imports files from minio
    Args:
        year (_type_): _description_

    Returns:
        _type_: _description_
    """
    s3_url = "https://" + os.environ["AWS_S3_ENDPOINT"]
    conn = s3fs.S3FileSystem(client_kwargs={'endpoint_url': s3_url})
    bucket = "radjerad/diffusion/inpi/"
    path2data = [bucket + "df_pm_all_years.csv",
                bucket +  "df_rep_all_years.csv"]
    
    with conn.open(path2data[0], mode="rb") as file_in:
        pm = pd.read_csv(file_in, sep=";")
    with conn.open(path2data[1], mode="rb") as file_in:
        representants = pd.read_csv(file_in, sep=";")
    return pm, representants

In [9]:
pm, rep = import_inpi()

/tmp/ipykernel_720/3992132590.py:18: DtypeWarning: Columns (7,8,10,11,16,30) have mixed types. Specify dtype option on import or set low_memory=False.
  pm = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_720/3992132590.py:20: DtypeWarning: Columns (7,15,17,25,26,32,33,34,35,36,37,38,39,40,41,42,43,45,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  representants = pd.read_csv(file_in, sep=";")


In [10]:
pm.shape

(5001845, 31)

In [11]:
rep.shape

(11142886, 51)